# 六、调用智谱 AI 生成 embedding


## 1. 调用智谱 AI Embedding API

智谱 AI 及 ChatGLM 在上一章已经进行了简单介绍，本章将介绍如何通过其获得文本的 embedding 的，以及将其封装成个性化 embedding 嵌入 LangChain 使用。关于什么是 embedding，具体作用为何，请参见第四部分《数据库搭建》。

In [3]:
import zhipuai
zhipuai.api_key = "your api key" #填写控制台中获取的 APIKey 信息

model = "text_embedding" #选择调用生成 embedding 的模型

自定义要生产 embedding 的文本

In [34]:
text = "要生成 embedding 的输入文本，字符串形式。每个输入不得超过模型的最大输入tokens数量512"

调用远程 API

In [27]:
response = zhipuai.model_api.invoke(
    model=model,
    prompt=text
)

官方的 SDK 返回的结果是字典的格式，这里我们可以直接传入键获得值。

我们通过查看 `code` 是否 == 200 来判断请求是否成功

In [28]:
print(response['code'])

200


返回的 embedding, token 等内容被存放在 `data` 中。我们可以查看生产 embedding 的长度

In [29]:
print(f"生成的 embedding 长度为: {len(response['data']['embedding'])}")

生成的 embedding 长度为: 1024


对于每次模型的调用来说，都是要消耗 token 的（花钱），token 的消耗都存放在 `data` 的 `usage` 中。 

一般情况下 ChatGLM 模型中 token 和字数的换算比例约为 1:1.8，但因为不同模型的分词不同，所以换算比例也存在差异，每一次实际处理 token 数量以模型返回为准。



我们可以查看输入的 tokens

In [30]:
print(f"用户输入的 tokens 数量为: {response['data']['usage']['prompt_tokens']}")

用户输入的 tokens 数量为: 28


In [36]:
print(f"用户输入的文本长度为: {len(text)}")

用户输入的文本长度为: 52


In [42]:
print(f"本次 token 和字数的换算比例为: {response['data']['usage']['prompt_tokens']/len(text)}, 和理论值 1:1.8 = {1/1.8} 接近")

本次 token 和字数的换算比例为: 0.5384615384615384, 和理论值 1:1.8 = 0.5555555555555556 接近


因为模型其实没有生成新的文本，只是获取到输入的 embedding，所以 token 数为0

In [31]:
print(f"模型输出的 tokens 数量为: {response['data']['usage']['completion_tokens']}")

模型输出的 tokens 数量为: 0


In [32]:
print(f"总 tokens 数量为: {response['data']['usage']['total_tokens']}")

总 tokens 数量为: 28


## 2. 使用 LangChain 调用智谱 AI Embedding

我们同样可以通过 LangChain 框架来调用智谱 AI 大模型，以将其接入到我们的应用框架中。

原生的 LangChain 是不支持智谱 AI Embedding 调用的，我们需要自定义一个 Embedding。

此处，我们可以直接调用已自定义好的 ZhipuAILLM

In [6]:
import zhipuai

In [4]:
from zhipuai_embedding import ZhipuAIEmbeddings

In [7]:
zhipuai.api_key = "xxx"    #输入自己的 api_key


In [8]:
zhipuai_embeddings = ZhipuAIEmbeddings(zhipuai_api_key=zhipuai.api_key)

我们可以生成 query 的 embedding。

In [10]:
query_embedding = zhipuai_embeddings.embed_query('你好')

print(query_embedding[:10])

[-0.13970163464546204, 0.04416792839765549, 0.020967043936252594, -0.19684536755084991, 0.08451296389102936, -0.0685092955827713, -0.05186789482831955, 0.11296232044696808, 0.12375720590353012, 0.1507660448551178]


我们也可以生成 doc_list 的 embedding。

In [ ]:
doc_list = [
    '你好',
    '什么是机器学习',
    '什么是深度学习',
    '什么是大模型'
]

doc_embeddings = zhipuai_embeddings.embed_documents(doc_list)

In [26]:
print('\n'.join([f'{doc} 的 embedding 为 {doc_embedding[:10]}...\n' for doc, doc_embedding in zip(doc_list, doc_embeddings)]))

你好 的 embedding 为 [-0.13970163464546204, 0.04416792839765549, 0.020967043936252594, -0.19684536755084991, 0.08451296389102936, -0.0685092955827713, -0.05186789482831955, 0.11296232044696808, 0.12375720590353012, 0.1507660448551178]...

什么是机器学习 的 embedding 为 [-0.04628180339932442, -0.09553179144859314, 0.010844158940017223, -0.1201983317732811, 0.16935010254383087, -0.15349552035331726, 0.17940732836723328, 0.1256963312625885, 0.09910263121128082, 0.1022590845823288]...

什么是深度学习 的 embedding 为 [-0.09552870690822601, -0.03116282820701599, 0.11060678958892822, 0.08925414085388184, 0.06898286193609238, -0.07559530436992645, 0.2021033763885498, -0.04418506473302841, 0.10697835683822632, -0.0666293203830719]...

什么是大模型 的 embedding 为 [0.10551410913467407, 0.1735556423664093, -0.24402201175689697, 0.02649446204304695, 0.09757085889577866, 0.030247822403907776, 0.4318920969963074, 0.06334380805492401, -0.02869655191898346, -0.1011139303445816]...

